In [20]:
import numpy as np
np.random.seed(42)

In [21]:
def loop_bootstrap(a: np.ndarray, bs_iters: int = 1_000, agg="mean", **kwargs) -> np.ndarray:
    """Return a sampling with replacement with given number of iterations

    Arguments:
    a:          Dataset to sample from
    bs_iters:   Number of sampl ing iteration
    agg:        Aggregation method to use, must be in ["mean", "median", "quantile"]
    Returns:
    np.ndarray with samples of size(bs_iters, len(a))
    """
    if agg not in ["mean", "median", "quantile"]:
        raise ValueError("agg should be in ['mean', 'median', 'quantile']")
    res = []
    for _ in range(bs_iters):
        if agg == "mean":
            res.append(np.mean(np.random.choice(a, len(a), replace=True)))
        elif agg == "median":
            res.append(np.median(np.random.choice(a, len(a), replace=True)))
        elif agg == "quantile":
            res.append(np.quantile(np.random.choice(a, len(a), replace=True)), q=kwargs["q"])
    return np.array(res)

def poisson_bootstrap(a:np.ndarray, bs_iters:int = 1_000):
    res = []
    for _ in range(bs_iters):
        poisson_weights = np.random.poisson(1, size=len(a))
        weights_sum = np.sum(poisson_weights)
        a_weighted = a * poisson_weights
        weighted_mean = np.sum(a_weighted) / weights_sum
        res.append(weighted_mean)
    return np.array(res)

def ci(diffs: np.ndarray, alpha: float = 0.05, **kwargs):
    # -> tuple[tuple[float, float], bool]:
    """Return a sampling with replacement with given number of iterations

    Arguments:
        diffs:  Dataset to compute percentiles upon
        alpha:  Confidence interval alpha
    Returns:
        tuple[CI left, CI right]
    """
    ci_l = np.percentile(diffs, (alpha / 2) * 100)
    ci_r = np.percentile(diffs, (1 - alpha / 2) * 100)
    return (ci_l, ci_r)

In [22]:
dataset = np.random.normal(10, 2, 100_000)

In [23]:
vanilla_bs = loop_bootstrap(dataset)
print('Vanilla bootstrap', ci(vanilla_bs))

Vanilla bootstrap (9.989096122954017, 10.014304152002024)


In [24]:
poisson_bs = poisson_bootstrap(dataset)
print('Poisson bootstrap', ci(poisson_bs))

Poisson bootstrap (9.989039863858277, 10.013947631627973)
